In [27]:
#Research references:
#1) Dry/wet cough classification: https://link.springer.com/article/10.1007/s10439-013-0741-6
#2) Pneumonia classification: https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=6987276

In [28]:
#from scipy.io.wavfile import read
#import wave
import numpy as np
import os
import sox
#import pywt #wavelets
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.utils import mediainfo
import matplotlib.pyplot as plt
import python_speech_features as spe_feats
import pandas as pd
from scipy.stats import kurtosis, skew, entropy
from scipy.signal import lfilter
import librosa
import math
import sys

## Settings

In [33]:
#Initialize data frame of features:

feats = pd.DataFrame([])

#tiny constant value
eps = sys.float_info.epsilon

#Features' settings:

fs_targ = 16000 # set all audios to this sampling frequency
n_channels_targ = 1

#framing
frame_len_s=0.025 #12 segments seemed adequeate in paper, since segments are no longer than 400ms (400ms/12=33.3ms)
frame_step_s=frame_len_s #according to paper: non-overlapping frames

frame_len = int(round(frame_len_s*fs_targ)) #in samples
frame_step = int(round(frame_step_s*fs_targ)) #in samples
win_func =np.hamming #at least for mfcc

#mfcc
cep_num= 13 #number of coefficients as in paper (https://link.springer.com/article/10.1007/s10439-013-0741-6)

#lp
lp_ord = int(round(2 + fs_targ/1000)) #standard rule of thumb for LP oder

#formants
nr_formants = 4 #as in paper, first 4 formants

## Functions

In [120]:
#Apply pre-emphasis (high-pass) filter
def apply_preEmph(x):
    x_filt = lfilter([1., -0.97], 1, x)
    return x_filt
        
#Obtain autocorrelation
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[int((result.size+1)/2):] #Note: other people use re.size/2:, but this does not work for me 
                                   # TODO: check consistency in other computers

#Compute zero-crossing rate
def get_zcr(x):
    zcr = (((x[:-1] * x[1:]) < 0).sum())/len(x)
    return zcr

#Compute log-energy
def get_logEnergy(x):
    logEnergy = np.log10( ( (np.power(x,2)).sum()/len(x) ) + eps)  
    return logEnergy

#Estimate fundamental frequency (F0)
def get_F0(x,fs):
    #autocorrelation-based method to extract F0
    xcorr_arr = autocorr(x)
    
    #looking for F0 in the frequency interval 50-500Hz, but we search in time domain
    min_ms = round(fs/500)
    max_ms = round(fs/50)
    
    xcorr_slot = xcorr_arr[max_ms+1:2*max_ms+1]
    xcorr_slot = xcorr_slot[min_ms:max_ms]
    t0 = np.argmax(xcorr_slot)
    F0 = fs/(min_ms+t0-1)
    return F0

#Estimate formants
def get_formants(x, lp_order, nr_formants):
    #LP-based method
    
    #compute lp coefficients
    a = librosa.lpc(x, lp_ord)

    #get roots from lp coefficients
    rts = np.roots(a)
    rts = [r for r in rts if np.imag(r) >= 0]

    #get angles
    angz = np.arctan2(np.imag(rts), np.real(rts))

    #get formant frequencies
    formants = sorted(angz * (fs_targ / (2 * math.pi)))
    
    return formants[0:nr_formants]

#Extract frequencies
def feature_extraction(x,fs,feats_df,lp_ord,ID,label):
#Extract features from signal x (identified as ID), and concatenate them to dataframe feats_df
#Features' reference: (see Appendix)
#[1]https://link.springer.com/article/10.1007/s10439-013-0741-6
#[2]https://espace.library.uq.edu.au/data/UQ_344963/s41943203_phd_submission.pdf?dsi_version=c5434db897ab74b192ca295a9eeca041&Expires=1585086202&Key-Pair-Id=APKAJKNBJ4MJBJNC6NLQ&Signature=c8k8DmG~KIxg0ToTO8rebm2MzHneCzJGkjSFRB7BYTEQ-MHXEr0ocHmISrldP3hFf9qmeiL11ezyefcNeRVeKIQ9PVjOl9pn7rXWcjA1o2voPn1VnDd8n7G2cT31apdj0LNMclhlXRPnCsGD66qDRqa3d-xaqqXhEqU73aw3ZgBgroO213MfJOqFhJxxXo2QEia0bSlDRTeX9KhSczFK-IFTPC6GwFL2L04por8pQRI3HF7E3f26O9zp9OhkwxSU9qfJah20WxZLA4PxREdv7JGoVBinR6T0mTcIaQi~B4IzYjSPSsTTADMNk5znVYIvSqgtMT~DY~qwlfq4SRdFjQ__
  
    
    #do features in a frame-basis
    x_frames = spe_feats.sigproc.framesig(x,frame_len,frame_step,win_func) #DOUBT: should I use window or not?
                                                                        #at least for formant estimation i should

    #0)Wavelets #TODO
    
    #HAMMING WINDOW USED IN MFCC TODO: CHECK IF USING THAT ONE
    #DOUBT: if log-energy feature is included, should I also include the first mfcc coefficient (c0) ?
    #1)mfcc
    mfcc_feat = spe_feats.mfcc(x,fs, winlen=frame_len_s,winstep=frame_step_s, numcep=cep_num,winfunc=win_func)
    
    #deltas to capture 
    mfcc_delta_feat = spe_feats.delta(mfcc_feat,1) #mfcc_delta_feat = np.subtract(mfcc_feat[:-1], mfcc_feat[1:]) #same
    mfcc_deltadelta_feat = spe_feats.delta(mfcc_delta_feat,1)          
    
    #2)zero-crossing rate
    zcr_feat = np.apply_along_axis(get_zcr, 1, x_frames)
    
    #3)Formant frequencies
    #using LP-coeffcs-based method
    #formant_feat = np.apply_along_axis(get_formants, 1, x_frames, lp_ord, nr_formants)
    
    #TODO: figure out a way to check beforehand the ill-coonditioning of the signal for lpc computing, 
    #until then, use full segment
    formant_feat = get_formants(x, lp_ord, nr_formants)
    
    #4)Log-energy
    logEnergy_feat =  np.apply_along_axis(get_logEnergy, 1, x_frames)
    
    #5)Pitch (F0)
    F0_feat =  np.apply_along_axis(get_F0, 1, x_frames,fs)
    
    #6)Kurtosis
    kurt_feat =  np.apply_along_axis(kurtosis, 1, x_frames)
    
    #7)Bispectrum Score (BGS)
    #TODO: see PhD thesis for more info on this feature
    
    #8)Non-Gaussianity Score (NGS)
    #TODO: see PhD thesis for more info on this feature
   
    #9) Adding skewness as measure of non-gaussianity (not in paper)
    skew_feat =  np.apply_along_axis(skew, 1, x_frames)
    
    #DOUBT: 10) Shannon entropy GETTING -inf in all cases, WHY??? Don't include until fixed
    #entropy_feat = entropy(x)
    #Maybe compute directly to check
    
    nr_frames = x_frames[1]
    
    #TODO: append matrices of features, one row per entry in dataframe (now it is done to append per segment)
    feats_df = feats_df.append(pd.DataFrame({'Id': ID, 'mfcc': [mfcc_feat], 'mfcc_d': [mfcc_delta_feat],
                                             'mfcc_dd': [mfcc_deltadelta_feat], 'kurtosis': kurt_feat,
                                             'logEnergy': logEnergy_feat, 'zcr': zcr_feat,
                                             'formants': [formant_feat], 'F0': F0_feat, 'skew': skew_feat,
                                             'label': label}, index=[0]), ignore_index=True, sort=False)
    return feats_df


## Main

In [121]:
norm_skip = False #skip normalization step (because it has been done previously)

#s = read(audiofile)
FOLDER_PATH = 'data/YT_set/wavs/1/'
for file_name in os.listdir(FOLDER_PATH):
    
    fname_noExt = os.path.splitext(file_name)[0] #file name without extension
    
    #full path file name
    full_fname = FOLDER_PATH+file_name
    #print(full_fname)
    
    #TODO: put normalized wavs in other folder
    #name for normalization
    NORM_FOLDER_PATH = 'data/YT_set/wavs/norm/'
    norm_fname = NORM_FOLDER_PATH + os.path.splitext(file_name)[0] + '_NORM.wav'
    
    if norm_skip is False: 
        ## Normalization
        
        #level to same dB
        tfm = sox.Transformer()
        tfm.gain(gain_db=0.0, normalize=False, limiter=False, balance=None)
        #downsample to 16kHz and 1 channel
        tfm.convert(samplerate=fs_targ, n_channels=n_channels_targ, bitdepth=None) 
        #tfm.norm(db_level=0.0)
    
        # create the output normalized audio
        
        print(norm_fname)
        tfm.build(full_fname, norm_fname)
        tfm.effects_log
    
    # load normalized audio
    s = AudioSegment.from_wav(norm_fname)
    #sampling rate:
    info = mediainfo(norm_fname)
    fs = float(info['sample_rate'])
    
    #get ID of recording
    ID = fname_noExt.split('-')[-2] #for the current type of naming
    print(file_name)
    print(ID)
    
    #get label
    label = fname_noExt.split('-')[-1] #for the current type of naming
    print(label)
    
    ## Segmentation of cough streams (silence-based)
    #min_silence_len in ms, silence_thresh in dB
    s_segments = split_on_silence (s, min_silence_len = 600, silence_thresh = -30)
    ## TODO: set more accurate thresholds, or find other way to split (variance-based?)
    
    #convert s_segments to numpy array format
    AudioSegment2numpy_arr = lambda x: np.asarray(x.get_array_of_samples())
    s_segments_np = list(map(AudioSegment2numpy_arr, s_segments))
    
    print('High-pass filtering...')
    #pre-emphasis filtering to each segment
    preEmph_filtering = lambda x: apply_preEmph(x)
    s_segments_filt = list(map(preEmph_filtering, s_segments_np))
    
    #TODO
    #2) the segment is divided into X non-overlapping subsegments (X=3 for dry/wet cough paper,
    #X=12 for pneumonia paper)
    #TODO: window framing: I think maybe the segments need to be windowed with non-overlapping frames? 
    #(see frame settings at beggining of notebook)
    
    print('Computing features...')
    #Feature extraction for each segment
    
    #(lambda function doesn't work )
    #feat_extr_step = lambda x, fs, feats_df, lp_ord, ID: feature_extraction(x,fs,feats_df,lp_ord,ID)
    #feats = feat_extr_step(s_segments_filt,fs,feats,lp_ord,ID)
    for idx, seg_i in enumerate(s_segments_filt):
        print(idx)
        feats = feature_extraction(seg_i,fs,feats,lp_ord,ID,label)
    
       

output_file: data/YT_set/wavs/norm/Another Girl Coughing-iYxUHA-Pwsk-Dry_NORM.wav already exists and will be overwritten on build


data/YT_set/wavs/norm/Another Girl Coughing-iYxUHA-Pwsk-Dry_NORM.wav
Another Girl Coughing-iYxUHA-Pwsk-Dry.wav
Pwsk
Dry
High-pass filtering...
Computing features...
0


ValueError: Shape of passed values is (25, 11), indices imply (1, 11)

In [6]:
feats

,Id,mfcc,mfcc_d,mfcc_dd,kurtosis,logEnergy,zcr,formants,F0,skew,label
0,Pwsk,"[[-36.04365338911715, 0.0, -3.2076317224819155...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",10.261335,7.033567,0.266776,"[615.4712130184763, 1009.0686183582355, 2031.2...",213.333333,-0.035899,Dry
1,Pwsk,"[[8.684765628904088, -33.314000680327545, -14....","[[0.104018338738749, 0.5568312860759939, 3.516...","[[-0.04379686556487439, 0.4181730602469935, 0....",7.269807,7.445741,0.352220,"[795.7480969526919, 900.8716953508034, 2145.66...",152.380952,-0.003134,Dry
2,Pwsk,"[[8.667678075801637, -29.540199480860924, -8.7...","[[1.9987420523521129, 9.192209889613906, -2.71...","[[0.8557744917363834, 2.4443792620245945, -0.0...",0.573200,6.289370,0.097091,"[551.2383040575314, 1045.736114981098, 1655.39...",163.265306,-0.180193,Dry
3,Pwsk,"[[8.866746402589161, -28.05970278797004, -3.95...","[[0.1494441638602355, -1.812811579600849, -0.7...","[[-0.16886290126833758, 0.3214850360255639, -2...",5.542743,7.493706,0.326300,"[715.756479741867, 1567.4161317150094, 1734.37...",326.530612,-0.019812,Dry
4,Pwsk,"[[9.691520844538447, -26.185043326491204, -1.3...","[[0.09306848338880691, -4.220576230914702, -6....","[[-0.10919975266958426, 0.24839989238782678, 0...",10.933372,7.087280,0.259046,"[558.7012952572038, 1242.9710489844333, 2103.9...",320.000000,-0.132976,Dry


In [103]:
x_frames = spe_feats.sigproc.framesig(s_segments_filt[0],frame_len,frame_step,win_func)

In [65]:
x_frames.shape

(25, 400)

In [106]:
np.linalg.cond(x_frames)

3.850969677588145e+17

In [104]:
formant_feat = np.apply_along_axis(get_formants, 1, x_frames, lp_ord, nr_formants)   

FloatingPointError: numerical error, input ill-conditioned?

In [102]:
formant_feat.shape

(30, 4)

### Pre-processing of features

In [ ]:
#TODO: Pre-proces the dataframe prior to use with a model (normalize if needed for the model, check there is no NaNs...)





## Model training

In [ ]:
#Cough sound
#Breathing rate
#Breathing rhytm (consistence smoothness)
#Cough rate
#Panic level
#Hoarseness